In [19]:
from craterdetection.matching.database import CraterDatabase
from craterdetection.matching.database import extract_robbins_dataset, load_craters
from craterdetection.matching.projective_invariants import crater_representation, matrix_adjugate

import numpy as np
import numpy.linalg as LA

In [2]:
lat, long, major, minor, psi, crater_id = extract_robbins_dataset(
    load_craters("../data/lunar_crater_database_robbins_2018.csv")
)

Wall time: 1.83 s


### Mathematical Representation of a Crater
#### Convert coordinates to 3D selenographic position

In [3]:
R_moon = 1737.1

p_Mi = R_moon*np.array([
    np.cos(lat)*np.cos(long),
    np.cos(lat)*np.sin(long),
    np.sin(lat)
]).T[:, :, None]

#### Define local ENU coordinate system

In [4]:
k = np.array([0, 0, 1])[:, None]
print(k.shape)

u_i = p_Mi / R_moon

e_i = np.cross(k.squeeze(), u_i.squeeze())[:, :, None]
e_i /= np.tile(LA.norm(e_i, ord=2, axis=(1,2)), (3, 1)).T[:, :, None]

n_i = np.cross(u_i.squeeze(), e_i.squeeze())[:, :, None]
n_i /= np.tile(LA.norm(n_i, ord=2, axis=(1,2)), (3, 1)).T[:, :, None]

T_EM = np.moveaxis(np.array([e_i, n_i, u_i]), 0, 1).squeeze()

(3, 1)


In [14]:
planes = np.concatenate([u_i, -R_moon*np.ones(p_Mi.shape[0])[:, None, None]], axis=1)
S = np.concatenate((np.identity(2), np.zeros((1, 2))), axis=0)

In [15]:
H_Mi = np.concatenate((T_EM @ S, p_Mi), axis=-1)

In [16]:
x_quadric = np.concatenate((H_Mi, np.tile(k.T, (p_Mi.shape[0], 1, 1))), axis=1)

In [17]:
C = crater_representation(0, 0, major, minor, psi)
C_star = matrix_adjugate(C)

In [18]:
Q_star = x_quadric @ C_star @ x_quadric.transpose(0, 2, 1)

### Homography and Action of a Projective Camera on a Crater Disk Quadric